# An introduction to neural networks with Keras
Florent Martin (Regensburg Universität)  
March 2018

Neural networks form one of the models used in machine learning.
What is machine learning?

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
sns.set()
plt.rc('axes',titlesize='xx-large')
plt.rc('axes',labelsize='x-large')
plt.rc('legend',fontsize='x-large')
%matplotlib inline

# import warnings
# warnings.filterwarnings("ignore")

# (Section 1) Before neural networks: Logistic regression

# 1.1 The iris dataset

In [ ]:
iris = sns.load_dataset("iris")
iris.head()

In [ ]:
sns.pairplot(iris, hue="species");

### Goal: knowing the petal width, predict if the iris is a virginica

# 1.2 Logistic regression

In [ ]:
iris['isVirginica'] = (iris['species'] == 'virginica').apply(int)
iris.head()

In [ ]:
fig, ax = plt.subplots()
iris.groupby('isVirginica').hist(column='petal_width', ax=ax, bins=15);
plt.legend(['not virginca', 'virginica']);

<font size=4>
The **logistic regression** algorithm searches for a function 
$$ P: [0,3] \to [0,1]$$
If $x = $ petal width, $P(x)$ is an estimate (= a guess) of the probability that the plant is a virginica.
</font>

<font size=4>
The idea of logistic regression is to look for a function of the form:
$$P_{w,b}(x) = \sigma(Wx+B)$$
where 
$$\sigma : \mathbb{R} \to [0,1]$$ is the **sigmoid function ** defined by 
$$ \sigma(x) = \frac{1}{1+e^{-x}}$$
We call $W$ the **weight** and $B$  the **bias**.
</font>

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [ ]:
x = np.arange(-10, 10, .01)
plt.plot(x, sigmoid(x))
plt.title('The sigmoid function');

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(18, 7))
ax1.set_title('Effect of B value in sigmoid')
for b in [-5, -2, 0, 2, 5]:
    ax1.plot(x, sigmoid(1 * x + b),label='B={}'.format(b))
    ax1.legend()

ax2.set_title('Effect of W value in sigmoid')
#for w in [0.05, .15, .3, .5, 1, 3]:
for w in [-1,0.05, .3, 3]:
    ax2.plot(x, sigmoid(w * x + 0),label='W={}'.format(w))
    ax2.legend()

## 1.3 Scikit-learn

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression()
model.fit(iris[['petal_width']], iris[['isVirginica']])

In [ ]:
petal_widths = np.arange(0,3,0.01)
predicted_proba = model.predict_proba(petal_widths.reshape(-1,1))[:,1]
plt.plot(petal_widths, predicted_proba, 'r--')
plt.xlabel('petal_width')
plt.ylabel('predicted probability');

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
iris.groupby('isVirginica').hist(column='petal_width', normed=True, ax=ax);
plt.legend(['not virginca', 'virginica'])
plt.plot(petal_widths, predicted_proba,'r--',label='predicted proba')
plt.hlines(0.5, *ax.get_xlim(), linestyles='dotted')
plt.vlines(petal_widths[np.argmax(predicted_proba > 0.5)], *ax.get_ylim(), linestyles='dotted');
petal_widths[np.argmax(predicted_proba > 0.5)]

<font size=4>
$$ \text{Accuracy} = \frac{\text{number of samples correctly classified}}{\text{total number of samples}}$$
<size>

In [ ]:
# with sklearn the method score returns the accuracy
model.score(iris[['petal_width']], iris[['isVirginica']])

### Graphical representation of logistic regression

![logistic regression](../reports/figures/01-log.png)

# 1.4 Keras

In [ ]:
from keras import regularizers, optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils.vis_utils import plot_model, model_to_dot
from IPython.display import SVG

In [ ]:
model = Sequential([
    Dense(1, input_dim=1),
    Activation('sigmoid'),
])

In [ ]:
model.compile(optimizer='sgd', 
                    metrics=['accuracy'],
                    loss='binary_crossentropy')

In [ ]:
model.fit(iris[['petal_width']], iris[['isVirginica']], epochs=500)

In [ ]:
model.metrics_names

In [ ]:
model.evaluate(iris[['petal_width']], iris[['isVirginica']])

In [ ]:
fig, ax = plt.subplots()
iris.groupby('isVirginica').hist(column='petal_width', normed=True, ax=ax);
plt.legend(['not virginica', 'virginica'])
predicted_proba = model.predict(petal_widths.reshape(-1,1))[:,0]
plt.plot(petal_widths, predicted_proba,'r--',label='proba')
plt.hlines(0.5, *ax.get_xlim(), linestyles='dotted')
plt.vlines(petal_widths[np.argmax(predicted_proba > 0.5)], *ax.get_ylim(), linestyles='dotted')

# (Section 2) Gradient descent

# 2.1 Optimization, gradient descent with Keras

In [ ]:
model = Sequential([
    Dense(1,input_dim=1,kernel_regularizer=regularizers.l2(.1)),
    Activation('sigmoid'),
])

In [ ]:
model.compile(optimizer=optimizers.SGD(lr=0.05), 
              loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
x_grid, y_grid = np.mgrid[-10:10:.1,-10:10:.1]

In [ ]:
def get_accuracy(weight,bias):
    layer =  model.layers[0]
    layer.set_weights( [ np.array([[weight]]) , np.array([bias]) ] ) 
    accuracy = model.evaluate(iris[['petal_width']], iris[['isVirginica']],verbose=0)[1]
    return accuracy

In [ ]:
vaccuracy = np.vectorize(get_accuracy)

In [ ]:
#takes two minutes to run
acc_grid = vaccuracy(x_grid,y_grid)

In [ ]:
# import pickle
# with open('../data/acc_grid', 'wb') as f:
#     # Pickle the 'data' dictionary using the highest protocol available.
#     pickle.dump(acc_grid, f)

In [ ]:
import pickle
with open('../data/acc_grid', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    acc_grid =pickle.load(f)

In [ ]:
fig,ax_acc = plt.subplots(figsize=(15,7))
plt.pcolor(x_grid , y_grid , acc_grid , cmap='RdBu_r')
plt.colorbar()
plt.title('Accuracy',fontsize='xx-large')
plt.xlabel('Weight (w)',fontsize='x-large')
plt.ylabel('Bias (b)',fontsize='x-large')
ax_acc.tick_params(labelsize='large')

## Problem: the accuracy is constant on huge zones

# 2.2 The cross entropy  loss function

<font size=4>
Fix the weigth $w$ and bias $b$. 
Define 
<br><br><br>
$$\mathcal{L}(p,y) = y \log(p) + (1-y)\log(1-p)$$
 <br>
$$\mathcal{L_{w,b}} = \sum_{i=1}^n y_i \log(p_i) + (1-y_i)\log(1-p_i)$$
<br>
where </font>

*  <font size=4>$y_i\in \{0,1\}$ is the actual class of the i-th sample</font>

*  <font size=4>$p_i \in [0,1]$ is the predicted probability $P_{w,b}(x_i)$ calculated by the logistic regression model for parameter values $w$ and $b$</font>

In [ ]:
probas = np.arange(0,1,.01)
loss0 = -np.log(1-pp)
loss1 = - np.log(pp)
plt.plot(probas,loss0,label='y = 0')
plt.plot(probas,loss1,label='y = 1')
plt.xlabel('proba')
plt.ylabel('Loss')
plt.title('Loss function:  $\mathcal{L}(p,y)$')
plt.legend();

In [ ]:
def initialise_keras_model(initial_weight = 9,initial_bias = 9,lr=0.05,reg=.1):
    model = Sequential()
    model.add(Dense(1,input_dim=1,kernel_regularizer=regularizers.l2(.1)))
    model.add(Activation('sigmoid'))
    sgd = optimizers.SGD(lr=lr)
    model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
    layer =  model.layers[0]
    layer.set_weights( [ np.array([[initial_weight]]) , np.array([initial_bias]) ] ) 
    return model

In [ ]:
def get_loss(weight,bias):
    layer =  model.layers[0]
    layer.set_weights( [ np.array([[weight]]) , np.array([bias]) ] ) 
    loss = model.evaluate(iris[['petal_width']], iris[['isVirginica']],verbose=0)[0]
    return loss

vloss = np.vectorize(get_loss)

In [ ]:
%time loss_grid = vloss(x_grid,y_grid)

In [ ]:
# import pickle
# with open('../data/loss_grid', 'wb') as f:
#     # Pickle the 'data' dictionary using the highest protocol available.
#     pickle.dump(loss_grid, f)

In [ ]:
import pickle
with open('../data/loss_grid', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    loss_grid =pickle.load(f)

In [ ]:
fig , ax_loss = plt.subplots(figsize=(20,7))
plt.pcolor(x_grid , y_grid , loss_grid , norm=colors.LogNorm() , cmap='RdBu_r')
plt.colorbar()
contour = plt.contour(x_grid, y_grid, loss_grid,20)
plt.title('Loss function')
plt.xlabel('weight')
plt.ylabel('bias')
model.layers[0].set_weights( [ np.array([[9]]) , np.array([9]) ] ) 
for i in range(30):
    old_weight, old_bias = model.get_weights()[0][0][0], model.get_weights()[1][0]
    model.fit(iris[['petal_width']], iris[['isVirginica']], epochs=5, verbose=0)
    weight, bias = model.get_weights()[0][0][0], model.get_weights()[1][0]
    plt.plot([old_weight,weight],[old_bias,bias],'kX--',markersize=8)

# (Section 3) Neural Networks: when linear methods are not sufficient

# 3.1 Versicolor

In [ ]:
iris['isVersicolor'] = (iris['species'] == 'versicolor').apply(int)
iris.head()

In [ ]:
fig, ax = plt.subplots()
iris.groupby('isVersicolor').hist(column='petal_width', ax=ax, bins=15);
plt.legend(['not versicolor', 'versicolor']);

## Goal: determine if an iris is a versicolor knowing its petal width

# 3.2 Let's try  logistic regression

In [ ]:
model = Sequential([
    Dense(1,input_dim=1,kernel_regularizer=regularizers.l2(.1)),
    Activation('sigmoid'),
])

In [ ]:
model.compile(optimizer=optimizers.SGD(lr=0.05), 
              loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(iris[['petal_width']], iris[['isVersicolor']], epochs=500)

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
iris.groupby('isVersicolor').hist(column='petal_width', normed=True, ax=ax);
plt.legend(['not versicolor', 'versicolor'])
predicted_proba = model.predict(petal_widths.reshape(-1,1))[:,0]
plt.plot(petal_widths, predicted_proba,'r--',label='proba')
plt.hlines(0.5, *ax.get_xlim(), linestyles='dotted')
plt.vlines(petal_widths[np.argmax(predicted_proba > 0.5)], *ax.get_ylim(), linestyles='dotted');

## BAD NEWS: PREDICTED PROBAS BY LOGISTIC REGRESSION CAN NOT UP AND DOWN

# 3.3 Add complexity to the neural network

In [ ]:
model = Sequential()
model.add(Dense(3, input_dim=1 ) )
model.add(Activation('sigmoid'))
model.add(Dense(1, input_dim=1 ) )
model.add(Activation('sigmoid'))
model.compile(optimizer=optimizers.SGD(lr=.1), loss='binary_crossentropy')

![NN](../reports/figures/02-hidden.png)

In [ ]:
plt.subplots(nrows=3,ncols=3,figsize=(18,12),sharex=True,sharey=True)
for i in range(1,10):
    plt.subplot(3,3, i)
    model.fit(iris[['petal_width']], iris[['isVersicolor']],epochs=150,verbose=0)
    probas = model.predict(petal_widths.reshape(-1,1))[:,0]
    plt.plot(petal_widths,probas,label='{} epochs'.format(i*150))
    plt.ylim((0,1))
    plt.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
iris.groupby('isVersicolor').hist(column='petal_width', normed=True, ax=ax);
plt.legend(['not versicolor', 'versicolor'])
predicted_proba = model.predict(petal_widths.reshape(-1,1))[:,0]
plt.plot(petal_widths, predicted_proba,'r--',label='proba')
plt.hlines(0.5, *ax.get_xlim(), linestyles='dotted')
plt.vlines(petal_widths[np.argmax(predicted_proba > 0.5)], *ax.get_ylim(), linestyles='dotted');